In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from functions import *

# 1 Folder Data loading
The code is loading multiple CSV files from a specified folder path and converting them into pandas DataFrames.

In [2]:
folder_path = "data/e production=ep sanyo ep sanyo 002" 
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

dataframes = []
dfarrnames = []

# Loop through the CSV files and load only those ending with "Format01=Kreis 5-064" and "TBA_CU"
for csv_file in csv_files:
    if "Format01=Kreis" in csv_file:
        if "TBA_Zyk" in csv_file:
            dfarr = pd.read_csv(csv_file, skiprows=[1])
            dfarr['Zeit'] = dfarr['Zeit'].apply(convert_to_linear_time)
            dfarr['Zeit'] = dfarr['Zeit'] - dfarr['Zeit'].iloc[0]
            dfarr['Zeit'] = dfarr['Zeit'] / 3600
            dataframes.append(dfarr)
            dfarrnames.append(csv_file)

noFiles = 0
for i, dfarr in enumerate(dataframes):
    num_rows, num_columns = dfarr.shape
    print(f"DataFrame {i + 1} - Rows: {num_rows}, Columns: {num_columns}")
    noFiles += 1
print(noFiles)

DataFrame 1 - Rows: 89410, Columns: 18
DataFrame 2 - Rows: 73298, Columns: 18
DataFrame 3 - Rows: 88666, Columns: 18
DataFrame 4 - Rows: 88428, Columns: 18
DataFrame 5 - Rows: 88300, Columns: 18
DataFrame 6 - Rows: 88156, Columns: 18
DataFrame 7 - Rows: 82806, Columns: 18
DataFrame 8 - Rows: 88059, Columns: 18
DataFrame 9 - Rows: 88009, Columns: 18
DataFrame 10 - Rows: 87786, Columns: 18
DataFrame 11 - Rows: 87437, Columns: 18
DataFrame 12 - Rows: 86887, Columns: 18
DataFrame 13 - Rows: 85831, Columns: 18
DataFrame 14 - Rows: 84802, Columns: 18
DataFrame 15 - Rows: 83475, Columns: 18
DataFrame 16 - Rows: 82491, Columns: 18
DataFrame 17 - Rows: 38664, Columns: 18
17


test that data is loaded succesfully

In [3]:
num_rows, num_columns = dataframes[1].shape

print(f"Number of rows (length): {num_rows}")
print(f"Number of columns (width): {num_columns}")
dataframes[0].head()

Number of rows (length): 73298
Number of columns (width): 18


,Schritt,Zustand,Zeit,Programmdauer,Schrittdauer,Zyklus,Zyklusebene,Prozedur,Prozedurebene,AhAkku,AhLad,AhEla,AhStep,Energie,WhStep,Spannung,Strom,Temp1
0,4,DCH,0.000000,193.0,3.0,1,1,TBA_Zyk,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,3.611689,0.000000,23.875
1,4,DCH,0.000008,232.0,42.0,1,1,TBA_Zyk,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,3.602915,-0.238044,23.875
2,4,DCH,0.000019,273.0,83.0,1,1,TBA_Zyk,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,3.485206,-3.369451,23.875
3,4,DCH,0.000019,273.0,83.0,1,1,TBA_Zyk,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,3.485206,-3.369451,23.875
4,4,DCH,0.000031,314.0,124.0,1,1,TBA_Zyk,0,-0.000076,0.0,0.000076,0.000076,-0.000268,0.000268,3.502387,-2.865025,23.875


Deleting Invalid data

In [4]:
valid_dataframes = []

for i,df in enumerate(dataframes):
    # Check if the 15th column is named 'Spannung'
    if len(df.columns) > 14 and df.columns[15] == 'Spannung':
        valid_dataframes.append(df)
    else:
        print('deleted : ',i)

# Update dataframes with valid dataframes
dataframes = valid_dataframes

noFiles = 0
for i, dfarr in enumerate(dataframes):
    num_rows, num_columns = dfarr.shape
    print(f"DataFrame {i + 1} - Rows: {num_rows}, Columns: {num_columns}")
    noFiles += 1
print(noFiles)

deleted :  7
DataFrame 1 - Rows: 89410, Columns: 18
DataFrame 2 - Rows: 73298, Columns: 18
DataFrame 3 - Rows: 88666, Columns: 18
DataFrame 4 - Rows: 88428, Columns: 18
DataFrame 5 - Rows: 88300, Columns: 18
DataFrame 6 - Rows: 88156, Columns: 18
DataFrame 7 - Rows: 82806, Columns: 18
DataFrame 8 - Rows: 88009, Columns: 18
DataFrame 9 - Rows: 87786, Columns: 18
DataFrame 10 - Rows: 87437, Columns: 18
DataFrame 11 - Rows: 86887, Columns: 18
DataFrame 12 - Rows: 85831, Columns: 18
DataFrame 13 - Rows: 84802, Columns: 18
DataFrame 14 - Rows: 83475, Columns: 18
DataFrame 15 - Rows: 82491, Columns: 18
DataFrame 16 - Rows: 38664, Columns: 18
16


### Crop The Values

from 3.65-3.85

In [35]:
I=0
for i, df in enumerate(dataframes):
    for zyclus_no in range(1, df['Zyklus'].max() + 1):
        filtered_df = df[(df['Zyklus'] == zyclus_no)]
        quarter_len = len(filtered_df) // 4
        filtered_df = filtered_df.iloc[quarter_len:]

        filtered_df_ahakku = filtered_df[(filtered_df['Spannung'] >= 3.65) & (filtered_df['Spannung'] <= 3.85)]
        filtered_df_max_ahakku = filtered_df.tail(1)

        # plot([15],filtered_df_ahakku)

        ahakku_values = filtered_df_ahakku['AhAkku'].values

        ahakku_array = np.array(ahakku_values)

        print('Total Data points' ,ahakku_array.size)
        ahakku_values
    


        # I+=1
    # print('Total data points : ',I)
    # print('\nfiltered_df_ahakku : ',filtered_df_max_ahakku['AhAkku'])
    # plot([15],filtered_df_ahakku)
    # print(filtered_df_ahakku['Spannung'],'\nfiltered_df_ahakku : ',filtered_df_max_ahakku['AhAkku'])
    #filtered_max_dataframes.append(filtered_df_ahakku)


Total Data points 53
Total Data points 47
Total Data points 47
Total Data points 47
Total Data points 47
Total Data points 47
Total Data points 48
Total Data points 46
Total Data points 48
Total Data points 46
Total Data points 47
Total Data points 46
Total Data points 46
Total Data points 46
Total Data points 46
Total Data points 46
Total Data points 46
Total Data points 45
Total Data points 47
Total Data points 45
Total Data points 48
Total Data points 46
Total Data points 45
Total Data points 47
Total Data points 46
Total Data points 46
Total Data points 48
Total Data points 45
Total Data points 46
Total Data points 47
Total Data points 46
Total Data points 47
Total Data points 45
Total Data points 47
Total Data points 46
Total Data points 45
Total Data points 45
Total Data points 44
Total Data points 44
Total Data points 44
Total Data points 45
Total Data points 46
Total Data points 43
Total Data points 45
Total Data points 43
Total Data points 46
Total Data points 45
Total Data po

Total Data points 43
Total Data points 43
Total Data points 43
Total Data points 42
Total Data points 42
Total Data points 42
Total Data points 41
Total Data points 42
Total Data points 42
Total Data points 42
Total Data points 42
Total Data points 41
Total Data points 42
Total Data points 41
Total Data points 42
Total Data points 40
Total Data points 40
Total Data points 40
Total Data points 41
Total Data points 40
Total Data points 41
Total Data points 40
Total Data points 41
Total Data points 40
Total Data points 40
Total Data points 39
Total Data points 39
Total Data points 39
Total Data points 39
Total Data points 39
Total Data points 39
Total Data points 40
Total Data points 39
Total Data points 39
Total Data points 39
Total Data points 39
Total Data points 39
Total Data points 39
Total Data points 39
Total Data points 39
Total Data points 38
Total Data points 38
Total Data points 38
Total Data points 38
Total Data points 38
Total Data points 38
Total Data points 38
Total Data po